# Flight Data Processing: 

The airport names, locations, and route distances and durations were collected from: https://github.com/Jonty/airline-route-data. The data from this repository is updated weekly from https://www.flightsfrom.com. 

### Key Assumptions:
* **Airport Check-in/Security:** 120 minutes - this is recommmended by most european airlines. (https://help.ryanair.com/hc/en-gb/sections/12502774638865-Estimated-Times-at-the-Airport, https://www.swedavia.com/arlanda/when-should-i-be-at-the-airport/)
* **City-to-Airport Travel:** the time is calculated for each city based on location of the city center and the airport with average of 40 km/hr - assuming car/taxi travel. This assumption is based on the method used in the "How fast are rail trips between EU cities and is rail faster than air" paper (https://ec.europa.eu/regional_policy/information-sources/publications/working-papers/2023/how-fast-are-rail-trips-between-eu-cities-and-is-rail-faster-than-air_en). This time is calculated for both departure and arrival airports and added to toal travel time. 

In [23]:
import json
import pandas as pd
import pathlib
import unicodedata as ud
from math import radians, cos, sin, asin, sqrt

data_path = pathlib.Path('data')
json_file = data_path / 'airline_routes.json'
city_filter_file = data_path / 'cities_filtered_500k_capitals.csv'

# assumptions
security_checkin_time = 120
avg_transit_speed = 40 #km/h

# countries
EU27 = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Czechia',
    'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary',
    'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
    'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden'
]

city_aliases= {
    # naming differences between datasets
    'nicosia': 'larnaca',
    'valletta': 'malta',
    'the hague': 'rotterdam',
    'leipzig': 'leipzig/halle',
    'hannover': 'hanover',
    
    # cities served by nearby airports
    'essen': 'dusseldorf',
    'duisburg': 'dusseldorf',
    'tilburg': 'eindhoven',
    
    # Polish l handling
    'wroclaw': 'wroclaw',
    'lodz': 'lodz'
}

# handles accents and case
def normalize(name):
    if not isinstance(name, str): return ""
    return "".join(c for c in ud.normalize('NFD', name)
                   if ud.category(c) != 'Mn').lower().strip()

# calculates km distance between two GPS points using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [float(lat1), float(lon1), float(lat2), float(lon2)])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# load in >500K populated cities list
df_filter = pd.read_csv(city_filter_file)
city_coords = {normalize(row['name_city']): (row['lat_city'], row['lng_city']) 
               for _, row in df_filter.iterrows()}
target_cities_norm = set(city_coords.keys())

# load flights data
with open(json_file, 'r') as f:
    air_data = json.load(f)

# find all of the valid airports that belong to the filtered cities
valid_iatas = {} 
airports_list = []

for iata, info in air_data.items():
    if info.get('country') in EU27:
        raw_city = info.get('city_name', '')
        norm_city = normalize(raw_city)
        
        matched_city_key = None
        if norm_city in target_cities_norm:
            matched_city_key = norm_city
        else:
            for target in target_cities_norm:
                if city_aliases.get(target) == norm_city:
                    matched_city_key = target
                    break
        
        if matched_city_key:
            c_lat, c_lng = city_coords[matched_city_key]
            dist_to_hub = haversine(c_lat, c_lng, info['latitude'], info['longitude'])
            
            # avg km/h assumption
            transit_mins = (dist_to_hub / avg_transit_speed) * 60
            
            valid_iatas[iata] = {
                'city_name': raw_city, 
                'transit_mins': transit_mins
            }
            
            airports_list.append({
                'city_name': raw_city,
                'airport_name': info.get('name'),
                'airport_code': iata,
                'longitude': info.get('longitude'),
                'latitude': info.get('latitude'),
                'dist_to_city_centroid_km': round(dist_to_hub, 2),
                'estimated_transit_mins': round(transit_mins, 2)
            })

# extract the routes only between those valid airports
routes_list = []
for iata, origin_info in valid_iatas.items():
    info = air_data[iata]
    for route in info.get('routes', []):
        dest_iata = route['iata']
        if dest_iata in valid_iatas:
            dest_meta = valid_iatas[dest_iata]
            flight_mins = route.get('min')
            
            # total = origin transit + security + flight + destination transit
            total_time = (origin_info['transit_mins'] + 
                          security_checkin_time + 
                          flight_mins + 
                          dest_meta['transit_mins'])

            routes_list.append({
                'from_city_name': origin_info['city_name'],
                'from_airport_code': iata,
                'to_city_name': dest_meta['city_name'],
                'to_airport_code': dest_iata,
                'distance_km': route.get('km'),
                'flight_duration_minutes': flight_mins,
                'total_journey_time_mins': round(total_time, 2)
            })

# create datasets and save them
df_airports = pd.DataFrame(airports_list)
df_routes = pd.DataFrame(routes_list).drop_duplicates(subset=['from_airport_code', 'to_airport_code'])

df_airports.to_csv(data_path / 'eu27_airports_filtered.csv', index=False)
df_routes.to_csv(data_path / 'eu27_flight_routes_filtered.csv', index=False)

print(f"Preprocessing Complete!")
print(f"Dataset 1 (Filtered Airports): {len(df_airports)} entries saved.")
print(f"Dataset 2 (Filtered Flight Routes): {len(df_routes)} entries saved.")

Preprocessing Complete!
Dataset 1 (Filtered Airports): 80 entries saved.
Dataset 2 (Filtered Flight Routes): 2265 entries saved.
